# this is Pixegami tutorial
### from this video :https://www.youtube.com/watch?v=tcqEUSNCn8I

In [ ]:
from dotenv import load_dotenv
import os
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import TextSplitter, RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores.chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from datetime import datetime
import json
import PyPDF2
import chromadb
chroma_client = chromadb.Client()

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
DIRECTORY_PATH='/Users/matansharon/python/chat_with_docs/AI_Apps/chat_with_txt/data'
DOCS_PATH='/Users/matansharon/python/chat_with_docs/AI_Apps/chat_with_txt/docs.json'

def load_and_read_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
    return text

def get_documents_list():
    
    with open(DOCS_PATH,'r') as f:
        data=json.load(f)
        docs=data['documents']
    return docs

def load_all_docs_in_data_folder():
    loader = DirectoryLoader(DIRECTORY_PATH)
    documents = loader.load()
    return documents

def split_text(text:str):
    
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100,
        length_function=len,
        add_start_index=True
    )
    chunks=text_splitter.split_text(text)
    return chunks

def create_new_db(chunks):
    
    path='chroma_db'
    if not os.path.exists(path):
        
        db=Chroma.from_texts(texts=[''],embedding=OpenAIEmbeddings(),persist_directory=path)
        return db
    return load_db()

def load_db():
    db = Chroma(persist_directory="chroma_db",embedding_function=OpenAIEmbeddings())
    return db

def get_results_with_scores(query,db):
    bar=0.5
    res=db.similarity_search_with_relevance_scores(query,k=3)
    
    return res
def get_prompt_template(results,query):
    template="""
    answer the question base only on the following context:
    {context}
    answer the question base on the above context: {query}
    
    """
    context_texts = []
    for i in range(len(results)):
        context_texts.append(results[i][0].page_content)
    temp = "\n\n---\n\n".join(context_texts)
    prompt_tamplate=ChatPromptTemplate.from_template(template)
    res=prompt_tamplate.format(context=temp,query=query)
    return res

def get_response(query,db,model):
    results=get_results_with_scores(query,db)
    prompt_template=get_prompt_template(results,query)
    response=model.invoke(prompt_template)
    return response.content

def main_app():
    db=create_new_db([''])
    docs=load_all_docs_in_data_folder()
    print(docs)
    return db
db=main_app()


In [ ]:
# len(chunks)

# model=ChatOpenAI()
# query='what is qlora?'

# response=get_response(query,db,model)
# print(response)
# db=create_new_db('')

In [ ]:
import chromadb.utils.embedding_functions as embedding_functions
import chromadb
from PyPDF2 import PdfReader
path='/Users/matansharon/python/chat_with_docs/data/pdf/qlora.pdf'
doc=PdfReader(path)
#get the name of the document
doc_name=path.split('/')[-1]

text=''
for page in doc.pages:
    text+=page.extract_text()



chunks=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100,length_function=len,add_start_index=True).split_text(text)




In [ ]:
print(get_response('who is the authors of the paper "Attention Is All You Need"',db,model))

In [ ]:
chat=ChatOpenAI(model='gpt-4-turbo-preview')


In [ ]:
print(chat.invoke(f'base on this: {content} what is qlora?'))